In [1]:
!pip install konlpy wordcloud scikit-learn

  Using cached numpy-1.22.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.0 kB)
INFO: pip is looking at multiple versions of matplotlib to determine which version is compatible with other requirements. This could take a while.
  Using cached matplotlib-3.9.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
  Using cached contourpy-1.2.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.8 kB)
Using cached numpy-1.22.4-cp39-cp39-macosx_11_0_arm64.whl (12.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.2 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.2.1-cp39-cp39-macosx_11_0_arm64.whl (244 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
  Attempting uninstall: contourpy━━━━━━━━━━━━━━━ 0/3 [numpy]
    Found existing installation

In [3]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from wordcloud import WordCloud

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

In [36]:
# CSV 불러오기
df = pd.read_csv('Dataset/jeju_review.csv')

# 특정 ID에 해당하는 리뷰만 추출 (예: id가 42인 경우)
target_id = 7843
target_reviews = df[df['id'] == target_id]['review'].dropna().astype(str).tolist()

In [38]:
# id별 리뷰 개수 집계
review_counts = df['id'].value_counts().reset_index()
review_counts.columns = ['id', 'review_count']

# 정렬된 결과 출력
print(review_counts.head(10))  # 상위 10개만 보고 싶다면

     id  review_count
0     2           100
1  8115           100
2  1970           100
3  8216           100
4  8209           100
5  8189           100
6  2020           100
7  2034           100
8  8097           100
9  1589           100


In [37]:
len(target_reviews)

97

In [7]:
# 3. 전처리 함수 정의
okt = Okt()

In [17]:
def preprocess(text):
    tokens = okt.pos(text, stem=True)  # 어간 추출 포함
    return [word for word, pos in tokens if pos in ['Noun', 'Adjective', 'Verb'] and len(word) > 1]

In [19]:
# 4. 벡터화 (BoW)
vectorizer = CountVectorizer(tokenizer=preprocess, max_df=0.9, min_df=10)
X = vectorizer.fit_transform(target_reviews)
vocab = vectorizer.get_feature_names_out()

/Users/phr/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
# 5. LDA 학습
lda = LatentDirichletAllocation(n_components=4, random_state=42)
lda.fit(X)

LatentDirichletAllocation(n_components=4, random_state=42)

In [21]:
# 6. 토픽별 키워드 출력
for idx, topic in enumerate(lda.components_):
    top_words = [vocab[i] for i in topic.argsort()[:-11:-1]]
    print(f"[Topic {idx}] → {', '.join(top_words)}")

[Topic 0] → 하다, 가다, 있다, 아웃백, 음식, 방문, 스테이크, 먹다, 같다, 않다
[Topic 1] → 하다, 직원, 받다, 생일, 샐러드, 주문, 아이, 처음, 해주다, 않다
[Topic 2] → 이다, 친절하다, 좋다, 하다, 맛있다, 서버, 방문, 아이, 담당, 음식
[Topic 3] → 먹다, 좋다, 맛있다, 메뉴, 파스타, 매장, 넓다, 아웃백, 스테이크, 투움바
